In [202]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 400
pd.options.display.max_columns = 400

In [326]:
def create_column_dictionary(text_file):
    '''
    Takes text file from User Guide to create library identifying locations of each feature.
    Returns pd.DataFrame
    The 'length' feature appears to be inconsistent
    '''

    import re
    import pandas as pd

    with open(text_file, 'r+') as f:
        ug = f.readlines()

    # Get field codes and corresponding columns
    field_list = [
        re.search(r'\b[0-9-]+\b\s\b[0-9]+\b\s\b[A-Z0-9_]{2,}\b',i).group() for i in ug if 
        re.search(r'\b[0-9-]+\b\s\b[0-9]+\b\s\b[A-Z0-9_]{2,}\b',i)
    ]

    # Place field numbers into DF
    field_code = pd.DataFrame(columns = ['start','end','length','field'])
    field_code['length'] = [re.split('\s',i)[1] for i in field_list]
    field_code['field'] = [re.split('\s',i)[2] for i in field_list]
    field_range = [re.split('\s',i)[0] for i in field_list]
    field_code['start'] = [re.split('-',i)[0] if re.search('-',i) else i for i in field_range]
    field_code['end'] = [re.split('-',i)[1] if re.search('-',i) else i for i in field_range]

    # Convert entry to numeric
    for i in ['start','end','length']:
        field_code[i] = pd.to_numeric(field_code[i])
    
    # Missing entries
    field_code_ug2018 = pd.DataFrame([[165,165,1,'F_FEDUC'], [280,281,2,'M_Ht_In'], 
                                      [292,294,3,'PWgt_R'], [299,301,3,'DWgt_R'], 
                                      [328,328,1,'f_RF_INFT'], [499,500,2,'OEGest_Comb'], 
                                      [501,502,2,'OEGest_R10'], [503,503,1,'OEGest_R3']], 
                                     columns = ['start','end','length','field'])
    
    field_code_ug2017 = pd.DataFrame([[280, 281, 2, 'M_Ht_In'],[292, 294, 3, 'PWgt_R'],[299, 301, 3, 'DWgt_R'],
                                      [328, 328, 1, 'f_RF_INFT'],[499, 500, 2, 'OEGest_Comb'],
                                      [501, 502, 2, 'OEGest_R10'],[503, 503, 1, 'OEGest_R3']],
                                     columns = ['start','end','length','field'])
    
    field_code_ug2016 = pd.DataFrame([[280, 281, 2, 'M_Ht_In'],[292, 294, 3, 'PWgt_R'],[299, 301, 3, 'DWgt_R'],
                                      [328, 328, 1, 'f_RF_INFT'],[499, 500, 2, 'OEGest_Comb'],
                                      [501, 502, 2, 'OEGest_R10'],[503, 503, 1, 'OEGest_R3']],
                                     columns = ['start','end','length','field'])
    
    field_code_ug2008 = pd.DataFrame([[609, 609, 1, 'F_LD_AUGMENT']],
                                     columns = ['start','end','length','field'])
    


    # Combine missing entries 
    if text_file.lower() == 'ug2018.txt':
        field_code = field_code.append(field_code_ug2018,ignore_index= True,sort = 'start')
        field_code = field_code.sort_values('start').reset_index().drop(columns='index')
    elif text_file.lower() == 'ug2017.txt':
        field_code = field_code.append(field_code_ug2017,ignore_index= True,sort = 'start')
        field_code = field_code.sort_values('start').reset_index().drop(columns='index')
    elif text_file.lower() == 'ug2016.txt':
        field_code = field_code.append(field_code_ug2016,ignore_index= True,sort = 'start')
        field_code = field_code.sort_values('start').reset_index().drop(columns='index')
    elif text_file.lower() == 'ug2008.txt':
        field_code = field_code.append(field_code_ug2008,ignore_index= True,sort = 'start')
        field_code = field_code.sort_values('start').reset_index().drop(columns='index')

    print(field_code.columns)
    

    # Find inconsistencies
    for i in range(0,len(field_code)-1):
        if field_code['end'][i] + 1 != field_code['start'][i+1]:
            print('check after: ', i,  field_code['field'][i], field_code['start'][i], '-', field_code['end'][i])
#         if field_code['end'][i] - field_code['start'][i] + 1 != field_code['length'][i]:
#             print('check range for: ', i,  field_code['field'][i], field_code['start'][i], '-', 
#                   field_code['end'][i], field_code['length'][i])

    

    return field_code

In [327]:
def no_more_filler(field_code):
    '''
    Looks for field that starts with 'FILLER' and drop the corresponding entry
    Returns shorter library pd.DataFrame
    '''
    import re
    
    field_code = field_code.loc[[False if re.search('^FILLER',i) else True for i in field_code['field']]]
    field_code.reset_index(inplace = True)
    field_code.drop(columns = ['index'], inplace = True)
    return field_code

In [328]:
def convert_to_csv(text_file, csv_file, column_dictionary):
    '''
    Pass fwf text, destination csv, column dictionary
    Read one line of the text file into python at a time
    Break each row apart into list and write onto csv
    '''

    import csv
    import time
    import re
    
    start_at = time.time()
    j=0
    c_dict = column_dictionary
    with open(csv_file,'w') as c:
        wc = csv.writer(c,quoting=csv.QUOTE_MINIMAL)
        
        # Write header
        wc.writerow(list(c_dict['field']))

        with open(text_file, 'r+') as f:
            for line in f:

#                 items = ["" if re.search(r'\A\s+\Z',line[c_dict['start'][i]-1 : c_dict['end'][i]]) 
#                          else line[c_dict['start'][i]-1 : c_dict['end'][i]] 
#                          for i in range(0,len(c_dict))]
                items = []
                for i in range(0,len(c_dict)):
#                     print(i)
                    from_here = c_dict['start'][i]-1
                    to_here = c_dict['end'][i]
                    if re.search(r'\A\s+\Z', line[from_here:to_here]):
                        items += [""]
                    else:
                        items += [line[from_here:to_here]]
#                     print(from_here, to_here)

                wc.writerow(items)
#         keep track of progress
                j += 1
                if j>20:
                    break
#                 if j%10000 == 0:
#                     print(j, time.time() - start_at)
    print('done at',j, 'in', time.time() - start_at)


In [329]:
# check similarities/differences between data structure for 2016-2018
u6 = create_column_dictionary('UG2016.txt')
u6 = no_more_filler(u6)
u7 = create_column_dictionary('UG2017.txt')
u7 = no_more_filler(u7)
u8 = create_column_dictionary('UG2018.txt')
u8 = no_more_filler(u8)
everything = pd.merge(pd.merge(u6,u7,how = 'outer', on = 'field', suffixes = ('_6','_7')),u8,how = 'outer', on = 'field')
everything['check'] = [everything['length_7'][i]==everything['length_6'][i]==everything['length'][i] for i in range(len(everything))]
everything[['field','length_6','length_7','length']][everything['check'] == False]

Index(['end', 'field', 'length', 'start'], dtype='object')
check after:  6 OCTERR 24 - 25
check after:  7 FILLER 24 - 31
check after:  20 MBCNTRY 80 - 81
check after:  23 MRCNTRY 85 - 86
check after:  24 FILLER 85 - 103
check after:  26 RCNTY 91 - 93
check after:  28 RCITY_POP 100 - 100
Index(['end', 'field', 'length', 'start'], dtype='object')
check after:  6 OCTERR 24 - 25
check after:  7 FILLER 24 - 31
check after:  20 MBCNTRY 80 - 81
check after:  23 MRCNTRY 85 - 86
check after:  24 FILLER 85 - 103
check after:  26 RCNTY 91 - 93
check after:  28 RCITY_POP 100 - 100
Index(['end', 'field', 'length', 'start'], dtype='object')
check after:  6 OCTERR 24 - 25
check after:  7 FILLER 24 - 31
check after:  20 FILLER 80 - 83
check after:  21 MBCNTRY 80 - 81
check after:  23 MRCNTRY 85 - 86
check after:  24 FILLER 85 - 103
check after:  26 RCNTY 91 - 93
check after:  28 RCITY_POP 100 - 100


C:\Users\pol\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,field,length_6,length_7,length
44,FBRACE,1.0,NaN,NaN
236,F_FEDUC,NaN,1.0,1.0
237,MHISPX,NaN,NaN,1.0
238,FHISPX,NaN,NaN,1.0


In [332]:
import re
non_reporting_col = list(filter(lambda i: re.search('\AF_',i), u8.field))

impute_flag_col = list(set(list(filter(lambda i: re.search('IMP',i), u8.field)) + ['LMPUSED'] 
                           + list(filter(lambda i: re.search('FLG',i), u8.field))))

no_col = list(filter(lambda i: re.search('\ANO_',i), u8.field))

terr_col = ['OCTERR','OCNTYFIPS','OCNTYPOP','MBCNTRY','MRCNTRY','MRTERR','RCNTY','RCNTY_POP','RCITY_POP','RECTYPE']

In [ ]:
# Find relationship between entries flagged as 'non-reporting'
import re

non_reporting = qwer[(qwer.iloc[:,[True if re.search('\AF_',i) else False for i in qwer.columns]]==0).all(axis=1)]

print(non_reporting[non_reporting_col].sum())

for i in non_reporting.columns:
    if non_reporting[i].nunique() <2:
        print(i, non_reporting[i].nunique())

non_reporting.groupby('RESTATUS').size()

In [ ]:
# Find relationship between 'unkown or not not stated' values and wholely 'not reported' codes
no_dict = {'NO_LBRDLV':'\ALD_', 'NO_RISKS':'\ARF_', 'NO_INFEC':'\AIP_', 'NO_MMORB':'\AMM_', 
 'NO_ABNORM':'\AAB_', 'NO_CONGEN':'\ACA_'}
import re

for k in no_dict.keys():
    print(qwer[qwer[k]==9].groupby(list(filter(lambda i: re.search(no_dict[k],i), qwer.columns))).size())
    for q in list(filter(lambda i: re.search(no_dict[k],i), qwer.columns)):
        print(q, qwer[qwer[q]=='U'][k].unique())
        
print(qwer.groupby(['NO_RISKS','RF_INFTR','RF_FEDRG','RF_ARTEC']).size())
print(qwer.groupby(['NO_LBRDLV','LD_INDL']).size())

In [330]:
u08 = create_column_dictionary('UG2008.txt')
u08 = no_more_filler(u08)
eights = pd.merge(u08,u8,how = 'outer', on = 'field', suffixes = ('_08','_18'))
eights['check'] = [eights['length_08'][i]==eights['length_18'][i] for i in range(len(eights))]
eights[['field','length_08','length_18']][eights['check']==False].sort_values('field')
eight_non_match=eights[['field','length_08','length_18']][
    eights['check']==False].sort_values('field').reset_index().drop(columns = ['index'])
eight_non_match['split'] = [re.split('',i)[1:-1] for i in eight_non_match['field']]

Index(['end', 'field', 'length', 'start'], dtype='object')


In [333]:
only08 = eight_non_match.loc[eight_non_match['length_08']>0][['field']].reset_index().drop(columns = ['index'])
only18 = pd.DataFrame(set(eight_non_match['field'])-set(only08.field)
                      - set(non_reporting_col) - set(impute_flag_col)-set(['DMAR'])- set(terr_col) - set(no_col),
                      columns=['field']).sort_values('field').reset_index().drop(columns = ['index'])
only08['split'] = [list(set(re.split('',i)[1:-1])) for i in only08['field']]
only18['split'] = [re.split('',i)[1:-1] for i in only18['field']]
only18['possibly'] = 0

In [305]:
only18

,field,split,possibly
0,APGAR10,"[A, P, G, A, R, 1, 0]",0
1,APGAR10R,"[A, P, G, A, R, 1, 0, R]",0
2,BFED,"[B, F, E, D]",0
3,BMI,"[B, M, I]",0
4,BMI_R,"[B, M, I, _, R]",0
5,CA_DOWN,"[C, A, _, D, O, W, N]",0
6,CIG0_R,"[C, I, G, 0, _, R]",0
7,CIG1_R,"[C, I, G, 1, _, R]",0
8,CIG2_R,"[C, I, G, 2, _, R]",0
9,CIG3_R,"[C, I, G, 3, _, R]",0


In [353]:
qaz = [[]]
for j in range(len(only18['field'])):
#     print(j)
    temp_list = dict()
    for i in range(len(only08)):
#         print(i)
        t_sum = sum([re.search(l,only18['field'][j]) != None for l in only08['split'][i]])
        t_den = len(only18['field'][j]) + len(only08['split'][i]) - sum(
            [re.search(l,only18['field'][j]) != None for l in only08['split'][i]])
#         print(t_sum/t_den)
        if t_sum/t_den >= 0.7:
            temp_list[only08['field'][i]] = t_sum/t_den

    if len(temp_list.keys())>0:
        print(temp_list.items())
        print('this', sorted(temp_list.keys(),key = temp_list.get))
        qaz += [[j]+sorted(temp_list.keys(),key = temp_list.get)]

# only18.drop(columns='split',inplace = True)

        
# only18['possibly']=[
#     [only08['field'][i],sum([re.search(l,only18['field'][j]) != None 
#                               for l in only08['split'][i]])/(len(only08['split'][i])+len(only18['split'][j])
#                                                              -sum([re.search(l,only18['field'][j]) != None 
#                                                                    for l in only08['split'][i]]))
#  for i in range(len(only08)) if sum([re.search(l,only18['field'][j]) != None 
#                                      for l in only08['split'][i]])/len(only08['split'][i])>0.7]
# for j in range(len(only18))]


dict_items([('CA_DOWNS', 0.875), ('F_UCA_DOWNS', 0.7), ('UCA_DOWNS', 0.7777777777777778)])
this ['F_UCA_DOWNS', 'UCA_DOWNS', 'CA_DOWNS']
dict_items([('UFHISP', 0.7142857142857143)])
this ['UFHISP']
dict_items([('FRACE', 0.7142857142857143), ('FRACEREC', 0.7142857142857143)])
this ['FRACE', 'FRACEREC']
dict_items([('FRACE', 0.7142857142857143), ('FRACEREC', 0.7142857142857143)])
this ['FRACE', 'FRACEREC']
dict_items([('FBRACE', 0.7142857142857143), ('FRACE', 0.8333333333333334), ('FRACEREC', 0.8333333333333334)])
this ['FBRACE', 'FRACE', 'FRACEREC']
dict_items([('ON_PROL', 0.7142857142857143)])
this ['ON_PROL']
dict_items([('F_LD_ANTIBIO', 0.7), ('LD_ANTI', 0.75)])
this ['F_LD_ANTIBIO', 'LD_ANTI']
dict_items([('UMHISP', 0.7142857142857143)])
this ['UMHISP']
dict_items([('MRACE', 0.7142857142857143), ('MRACEREC', 0.7142857142857143)])
this ['MRACE', 'MRACEREC']
dict_items([('MRACE', 0.7142857142857143), ('MRACEREC', 0.7142857142857143)])
this ['MRACE', 'MRACEREC']
dict_items([('MBRACE', 

In [354]:
qaz

[[],
 [5, 'F_UCA_DOWNS', 'UCA_DOWNS', 'CA_DOWNS'],
 [14, 'UFHISP'],
 [16, 'FRACE', 'FRACEREC'],
 [17, 'FRACE', 'FRACEREC'],
 [18, 'FBRACE', 'FRACE', 'FRACEREC'],
 [22, 'ON_PROL'],
 [32, 'F_LD_ANTIBIO', 'LD_ANTI'],
 [35, 'UMHISP'],
 [42, 'MRACE', 'MRACEREC'],
 [43, 'MRACE', 'MRACEREC'],
 [44, 'MBRACE', 'MRACE', 'MRACEREC'],
 [47, 'OP_ECVF', 'F_OB_CERVIC'],
 [48, 'OP_ECVS'],
 [53, 'PRECARE_REC'],
 [55, 'UPREVIS'],
 [61, 'F_URF_CHYPER', 'URF_CHYPER', 'RF_PHYP', 'F_URF_PHYPER', 'URF_PHYPER'],
 [63, 'URF_DIAB', 'RF_DIAB'],
 [64,
  'F_URF_CHYPER',
  'URF_CHYPER',
  'RF_PHYP',
  'F_URF_PHYPER',
  'URF_PHYPER',
  'RF_GHYP'],
 [66, 'URF_DIAB', 'RF_DIAB'],
 [67, 'F_URF_CHYPER', 'URF_CHYPER', 'RF_PHYP', 'F_URF_PHYPER', 'URF_PHYPER']]